<a href="https://colab.research.google.com/github/cnndabbler/agentic-workflows/blob/main/run_pydantic_ai_with_Ollama_and_DeepSeek_R1_Step_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pydantic Agents running DeepSeek-R1 using Ollama

In [1]:
!pip install -q pydantic_ai
!pip install -q pydantic
!pip install -q rich

# Initiate Ollama and load the models

In [2]:
import subprocess
import time

# Run the Ollama installation script
subprocess.run("curl -fsSL https://ollama.com/install.sh | sh", shell=True, check=True)

# Start the Ollama server in the background
ollama_process = subprocess.Popen(["ollama", "serve"])

# Allow some time for the server to initialize
time.sleep(5)

# Reasoner models
# !ollama pull deepseek-r1:32b > /dev/null 2>&1
# !ollama pull deepseek-r1:32
# !ollama pull deepseek-r1:14b > /dev/null 2>&1
!ollama pull deepseek-r1:14b

# Parser models
# !ollama pull granite3.1-dense > /dev/null 2>&1
!ollama pull granite3.1-dense



pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 6e9f90f02bb3... 100% ▕▏ 9.0 GB                         
pulling 369ca498f347... 100% ▕▏  387 B                         
pulling 6e4c38e1172f... 100% ▕▏ 1.1 KB                         
pulling f4d24e9138dd... 100% ▕▏  148 B                         
pulling 3c24b0c80794... 100% ▕▏  488 B                         
verifying sha256 digest 
writing manifest 
success 
pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠇ pulling manifest 
pulling 44d19d212d76... 100% ▕▏ 5.0 GB                         
pulling f76a906816c4... 100% ▕▏ 1.4 KB                         
pulling f7b956e70ca3... 100% ▕▏   69 B                         
pulling 492069a62c25... 100% ▕▏  11 KB                         
pulling f9cd69f4077d... 100% ▕▏ 

In [3]:
!ollama list

NAME                       ID              SIZE      MODIFIED               
granite3.1-dense:latest    34d3be74ec54    5.0 GB    Less than a second ago    
deepseek-r1:14b            ea35dfe18182    9.0 GB    1 second ago              


# Setup Ollama Models for Pydantic Agents

In [4]:
# Initialize models

from pydantic_ai.models.openai import OpenAIModel

# Ollama with DeepSeekR1
deepseek_reasoner_model = OpenAIModel(
    # 'deepseek-r1:32b',
    'deepseek-r1:14b',
    base_url="http://localhost:11434/v1",
    api_key="whatever",
)

# Ollama with Granite
granite_parser_model = OpenAIModel(
    # 'qwen2.5',
    'granite3.1-dense',
    base_url="http://localhost:11434/v1",
    api_key="whatever",
)

# Gemini
from google.colab import userdata
import os

# Get the API key from Google Colab userdata
api_key = userdata.get('GOOGLE_API_KEY')

# Set the API key as an environment variable, which `pydantic-ai` will automatically use for Gemini
os.environ['GEMINI_API_KEY'] = api_key

gemini_parser_model = "google-gla:gemini-1.5-flash"

# Pydantic Classes

In [5]:
from typing import List
from pydantic import BaseModel, Field, confloat
import asyncio
from pydantic_ai import Agent, RunContext
from rich.console import Console
from rich.table import Table
from rich import box

# Define the Joke model
class Joke(BaseModel):
    setup: str = Field(..., description="The setup part of the joke")
    punchline: str = Field(..., description="The punchline of the joke")
    category: str = Field(..., description="Category of the joke (e.g., 'pun', 'wordplay', 'dad joke')")
    thinking: str = Field(..., description="the thought process and planning captured between <think> tags without any change")


In [6]:
# Create Joke Creator Agent
joke_creator = Agent(
    model=deepseek_reasoner_model,
    result_type=str,
    deps_type=str,  # Takes a string prompt as input
    system_prompt=(
        "You are a joke creator that specializes in creating funny, family-friendly dad jokes. "
        "Your output MUST follow this EXACT format with EXACT field names:\n\n"
        "Setup: [setup text]\n"
        "Punchline: [punchline text]\n"
        "Category: [category]\n"
    ),
)

# Binding to Pydantic Joke class: Failure

- You will see the following error:
- BadRequestError: Error code: 400 - {'error': {'message': 'registry.ollama.ai/library/deepseek-r1:32b does not support tools', 'type': 'api_error', 'param': None, 'code': None}}

In [7]:
# Create Joke Creator Agent binding result_type to Joke class
joke_creator_with_pydantic_class = Agent(
    model=deepseek_reasoner_model,
    result_type=Joke,  # We'll attempt to parse into our Pydantic Joke model
    deps_type=str,  # Takes a string prompt as input
    system_prompt=(
        "You are a joke creator that specializes in creating funny, family-friendly dad jokes. "
        "Your output MUST follow this EXACT format with EXACT field names:\n\n"
        "Setup: [setup text]\n"
        "Punchline: [punchline text]\n"
        "Category: [category]\n"
    ),
)

NOTE: Only run the cell below to verify that Pydantic Agent does not support DeepSeek-R1

In [8]:
# joke_text = await joke_creator_with_pydantic_class.run("Create a dad joke")
# print("\nRaw joke text:")
# print("-" * 50)
# print(joke_text.data)
# print("-" * 50)

# Step 1: Create Joke with DeepSeek-R1

In [9]:
# Step 1: Create raw joke text using DeepSeek-R1
print("Creating joke...")
joke_text = await joke_creator.run(
    "Create a dad joke",
    model_settings={"max_tokens": 2000}
    )
if not joke_text:
    raise ValueError("Failed to create joke")
print("\nRaw joke text:")
print("-" * 50)
print(joke_text.data)
print("-" * 50)

Creating joke...

Raw joke text:
--------------------------------------------------
<think>
Okay, so I need to create a joke that's both funny and family-friendly for dads. First, I should think about common themes in dad jokes. Animals are always a safe and funny topic because they're relatable and easy to make puns with.

I remember some animal-related setups, but I want something fresh. Maybe using an animal activity as the setup. Let me consider different animals: dogs, cats, birds... Oh, how about a penguin? Penguins do waddle, which is inherently funny.

The waddling could lead to a Dad joke about walking or falling over, maybe in a grocery store. That scenario is amusing because it's unexpected—penguins don't usually stroll through grocery stores!

Now for the punchline. It should tie the waddling back to the grocery shopping scenario with a humorous twist. Something like asking about balance would be perfect because penguins might not be too graceful on land, and dads can relat

# Step 2A : Parser using Gemini

In [10]:
# Create Parser Agent with 3 retries max
joke_parser_gemini = Agent(
    model=gemini_parser_model,
    result_type=Joke,  # We'll parse into our Pydantic Joke model
    retries=3, # Maximum retries
    # deps_type=str,
    system_prompt=(
        "You are a joke parser that extracts components from the input text to create a Joke object. "
        "Your ONLY task is to find and extract these components from the input text without making any change:\n"
    )
)

# Parse the text into a Pydantic model using Gemini
print("\nParsing joke...")
parsed_joke = await joke_parser_gemini.run(
    joke_text.data)
if not parsed_joke:
    raise ValueError("Failed to parse joke")
print("\nParsed joke text:")
print("-" * 50)
print(parsed_joke.data)
print("-" * 50)

# Step 3: Display results
console = Console()

# Create assessment table
table = Table(title="Joke Parsed", box=box.ROUNDED)
table.add_column("Property", style="cyan", width=20)
table.add_column("Value", style="yellow")

# Add joke content
table.add_row("Setup", parsed_joke.data.setup)
table.add_row("Punchline", parsed_joke.data.punchline)
table.add_row("Category", parsed_joke.data.category)
table.add_row("","")


# Add thinking process
table.add_row("", "")
table.add_row("[bold]Thinking Process", "")
table.add_row("", parsed_joke.data.thinking)

# Print the table
console.print("\n")
console.print(table)


Parsing joke...

Parsed joke text:
--------------------------------------------------
setup="Why did the penguin get invited to the human's house?" punchline='Because they heard he was great at grocery store waddles… and could always find the balance!' category='Animal' thinking="\\nOkay, so I need to create a joke that's both funny and family-friendly for dads. First, I should think about common themes in dad jokes. Animals are always a safe and funny topic because they're relatable and easy to make puns with.\\n\\nI remember some animal-related setups, but I want something fresh. Maybe using an animal activity as the setup. Let me consider different animals: dogs, cats, birds... Oh, how about a penguin? Penguins do waddle, which is inherently funny.\\n\\nThe waddling could lead to a Dad joke about walking or falling over, maybe in a grocery store. That scenario is amusing because it's unexpected—penguins don't usually stroll through grocery stores!\\n\\nNow for the punchline. It sho

                                                    Joke Parsed                                                    
╭──────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────╮
│ Property             │ Value                                                                                    │
├──────────────────────┼──────────────────────────────────────────────────────────────────────────────────────────┤
│ Setup                │ Why did the penguin get invited to the human's house?                                    │
│ Punchline            │ Because they heard he was great at grocery store waddles… and could always find the      │
│                      │ balance!                                                                                 │
│ Category             │ Animal                                                                                   │
│                      │                                                                                          │
│                      │                                                                                          │
│ Thinking Process     │                                                                                          │
│                      │ \nOkay, so I need to create a joke that's both funny and family-friendly for dads.       │
│                      │ First, I should think about common themes in dad jokes. Animals are always a safe and    │
│                      │ funny topic because they're relatable and easy to make puns with.\n\nI remember some     │
│                      │ animal-related setups, but I want something fresh. Maybe using an animal activity as the │
│                      │ setup. Let me consider different animals: dogs, cats, birds... Oh, how about a penguin?  │
│                      │ Penguins do waddle, which is inherently funny.\n\nThe waddling could lead to a Dad joke  │
│                      │ about walking or falling over, maybe in a grocery store. That scenario is amusing        │
│                      │ because it's unexpected—penguins don't usually stroll through grocery stores!\n\nNow for │
│                      │ the punchline. It should tie the waddling back to the grocery shopping scenario with a   │
│                      │ humorous twist. Something like asking about balance would be perfect because penguins    │
│                      │ might not be too graceful on land, and dads can relate to that joke.\n\nPutting it all   │
│                      │ together:\n\nSetup: Why did the penguin get invited to the human's house?\nPunchline:    │
│                      │ Because they heard he was great at grocery store waddles… and could always find the      │
│                      │ balance!\n\nThis works because it's simple, has a pun, is clear, and family-friendly.    │
│                      │ It's easy to understand and gives dad a light-hearted moment.\n                          │
╰──────────────────────┴──────────────────────────────────────────────────────────────────────────────────────────╯

# Step 2B: Ollama with Granite Model Parser

In [11]:
# Create Parser Agent with 10 retries max
joke_parser_ollama = Agent(
    model=granite_parser_model,
    result_type=Joke,  # We'll parse into our Pydantic Joke model
    retries=10,
    # deps_type=str,
    system_prompt=(
        "You are a joke parser that extracts components from the input text to create a Joke object. "
        "Your ONLY task is to find and extract these components from the input text without making any change:\n"
    )
)

# Parse the text into a Pydantic model using Ollama Granite
print("\nParsing joke...")
parsed_joke = await joke_parser_ollama.run(
    joke_text.data)
if not parsed_joke:
    raise ValueError("Failed to parse joke")
print("\nParsed joke text:")
print("-" * 50)
print(parsed_joke.data)
print("-" * 50)

# Step 3: Display results
console = Console()

# Create assessment table
table = Table(title="Joke Parsed", box=box.ROUNDED)
table.add_column("Property", style="cyan", width=20)
table.add_column("Value", style="yellow")

# Add joke content
table.add_row("Setup", parsed_joke.data.setup)
table.add_row("Punchline", parsed_joke.data.punchline)
table.add_row("Category", parsed_joke.data.category)
table.add_row("","")


# Add thinking process
table.add_row("", "")
table.add_row("[bold]Thinking Process", "")
table.add_row("", parsed_joke.data.thinking)

# Print the table
console.print("\n")
console.print(table)


Parsing joke...


UnexpectedModelBehavior: Exceeded maximum retries (10) for result validation